## Memecahkan Sistem persaman menggunakan Eliminasi


Pada bagian ini kita membahas kode yang diperlukan untuk menyelesaikan sistem linier $AX=B$ menggunakan eliminasi. Kita akan membatasi pada kasus di mana $A$ adalah matriks bujur sangkar $n\times n$, dan sistem mempunyai tepat satu solusi. Kasus yang lebih umum memerlukan lebih banyak pengetahuan tentang teori yang mendasarinya dan akan dibahas pada bab selanjutnya.

Saat menulis kode program untuk mengerjakan soal yang kompleks, dimulai dengan   membuat kode program untuk  mengerjakan bagian-bagian yang kecil  yang lebih sederhana dari masalah komplek tersebut.Setelah itu baru merakit bagian-bagian tersebut untuk memecahkan masalah yang lebih kompleks. Dalam kasus kami, kami akan membagi metode solusi penyelesaian  menjadi dua bagian.

1. Melakukan eliminasi pada matriks augmented yang  terkait.
2. Lakukan substitusi balik  pada sistem segitiga yang dihasilkan dari eliminasi.

Penting juga untuk dipertimbangkan  bagaimana  menulis kode  sehingga kita dapat menggunakannya kembali untuk tugas lain nantinya.

### Fungsi subsistusi balik (Back substitution function)

Kita akan mulai dengan langkah, karena bagian tersebut lebih mudah. JIka tahap eliminasi telah berhasil, kita akan memilik sistem  segitiga atas  $UX=B$ yang memiliki bentuk seperti berikut
$$
\begin{equation}
\left[ \begin{array}{rrrr} * & * & * & * \\ 0 & * & * & * \\ 0 & 0 & * & * \\ 0 & 0 & 0 & * \end{array}\right]
\left[ \begin{array}{r}  x_1 \\  x_2  \\ x_3 \\ x_4  \end{array}\right]=
\left[ \begin{array}{r}  * \\  *  \\ * \\ *  \end{array}\right]
\end{equation}
$$

Kode tersebut akan kita masukkan ke dalam suatu fungsi agar nantinya mudah digunakan kembali. Untuk fungsi ini, misalkan kita diberikan matriks segitiga atas $U$ dan vektor yang diketahui $B$ dan kita ingin mencari vektor $X$ sehingga $UX=B$. Perhatikan bahwa kita dapat membuat asumsi lain, seperti matriks $U$ yang memiliki elemen-elemen diagonal sama dengan 1. Semakin sedikit asumsi yang kita buat, semakin berguna kode tersebut nantinya.

In [ ]:
import numpy as np

def BackSubstitution(U,B):
# =============================================================================
#     U is a NumPy array that represents an upper triangular square mxm matrix.
#     B is a NumPy array that represents an mx1 vector
#     BackSubstitution will return an mx1 vector that is the solution of the
#     system UX=B.
# =============================================================================
    m = U.shape[0]  # m is number of rows and columns in U
    X = np.zeros((m,1))

    for i in range(m-1,-1,-1):  # Calculate entries of X backward from m-1 to 0
        X[i] = B[i]
        for j in range(i+1,m):
            X[i] -= U[i][j]*X[j]
        if (U[i][i] != 0):
            X[i] /= U[i][i]
        else:
            print("Zero entry found in U pivot position",i,".")
    return X

Sebelum melanjutkan, mari kita uji fungsi ini. Kita dapat membangun matriks dengan bentuk segitiga yang tepat, *pilih solusi*, lalu buat sistem $UX=B$ sehingga kita mengetahui solusinya.

In [ ]:
# Make up an upper triangular matrix.  Could we make a random upper triangular matrix here?  What could go wrong?
U = np.array([[3,0,1],[0,1,-1],[0,0,-3]])
# We will choose the solution X_true.  We can put in any numbers we like here.
X_true = np.array([[3],[4],[3]])
B = U@X_true
# Call the function.  It should produce the same array as X_true if it works correctly.
X = BackSubstitution(U,B)

print(X)

[[3.]
 [4.]
 [3.]]


[link text](https://)Sebelum membahas langkah eliminasi, kita harus mencatat bahwa fungsi $\texttt{BackSubstitution}$ ini *akan gagal* memberikan hasil yang berarti jika salah satu entri diagonal $U$ adalah nol. Kita harus mengingat hal ini saat menggunakan kode ini di masa mendatang.

### Fungsi reduksi baris

Eliminasi adalah bagian yang lebih besar dan lebih kompleks dari metode penyelesain. Ini juga merupakan tugas umum yang akan muncul di bagian selanjutnya, jadi kita memerlukan beberapa kode yang dapat kita gunakan kembali di lain waktu. Kami menginginkan fungsi yang akan menjalankan semua langkah eliminasi. Idealnya kita ingin fungsi tersebut melakukan eliminasi pada array dengan ukuran atau dengan ukuran matrik sembarang, dan juga dapat  melakukan pertukaran baris bila diperlukan.

Mari kita perjelas tujuannya. Fungsi tersebut harus menerima array matrik sembarang dan menghasilkan array yang memiliki karakteristik sebagai berikut.

1. Elemen bukan nol pertama di setiap baris adalah 1. Elemen ini disebut pivot.
2. Setiap pivot terletak di sebelah kanan pivot pada semua baris di atasnya.
3. Elemen di bawah setiap pivot adalah 0.
4. Baris yang semuan elemennya nol terletak di bawah baris yang berisi entri bukan nol.

Matriks seperti ini dikatakan dalam **bentuk eselon baris**. Berikut tiga contoh matriks dalam bentuk yang memenuhi kriteria diatas.

   
$$
\begin{equation}
\left[ \begin{array}{cccc} 1 & * & * & * \\ 0 & 1 & * & * \\ 0 & 0 & 1 & * \end{array}\right]
\end{equation}
$$

$$
\begin{equation}
\left[ \begin{array}{ccc} 1 & * & *  \\ 0 & 0 & 1  \\ 0 & 0 & 0 \end{array}\right]
\end{equation}
$$


$$
\begin{equation}
\left[ \begin{array}{cccccc} 1 & * & * & * & * & * \\ 0 & 0 & 1 & * & * & * \\ 0 & 0 & 0 & 1 & * & * \end{array}\right]
\end{equation}
$$

Penting untuk diperhatikan bahwa setiap baris dapat berisi paling banyak satu pivot dan setiap kolom dapat berisi paling banyak satu pivot.

Sebelum membuat kode untuk mencari bentuk eselon baris suatu matriks, terlebih dahulu kita klarifikasi terlebih dahulu  matriks yang kita gunakan untuk menyelesaikan sistem berbentuk $AX=B$. Ingat pembahasan pada  bagian [Eliminasi Gaussian](Gaussian_Elimination.ipynb) bahwa operasi baris yang sama diperlukan untuk membawa $A$ ke bentuk eselon baris juga harus diterapkan ke $B$. Dalam praktiknya kita dapat menggabungkan $A$ dan $B$ untuk membentuk apa yang kita sebut **matriks augmented**. Selanjutnya kit melakukan operasi baris pada matriks ini. Berikut adalah contoh matriks augmented , yang akan kita tulis dengan $[A|B]$, yang terkait dengan sistem $AX=B$.

$$
\begin{equation}
AX = B \hspace{1cm} \left[ \begin{array}{rrr} 3 & -1 \\ 5 & 2 \end{array}\right]
\left[ \begin{array}{r} x_1 \\ x_2 \end{array}\right]=
\left[ \begin{array}{r} 0\\ 7  \end{array}\right] \hspace{1cm} \to \hspace{1cm}
[A|B] = \left[ \begin{array}{rr|r} 3 & -1 & 0 \\ 5 & 2 & 7 \end{array}\right]
\end{equation}
$$


Disini ukuran $A$ adalah $n\times n$, yang berarti matriks augmented yang perlu kita proses adalah $n\times(n+1)$. Kami juga akan berasumsi bahwa sistem $AX=B$ memiliki penyelesaian tunggal. Jika hal ini benar, matriks augmented akan memiliki pivot di masing-masing kolom $n$ pertama, dengan posisi pivot terletak di sepanjang garis element diagonal yang dimulai dari elemen kiri atass. Jika bentuk eselon baris dari matriks augmented memiliki nilai nol di salah satu posisi ini, proses penyelesaian terpecah seperti yang ditunjukkan dalam contoh [Eliminasi Gaussian](Gaussian_Elimination.ipynb).


In [ ]:
import laguide as lag

def RowReduction(A):
# =============================================================================
# A is a NumPy array that represents an augmented matrix of dimension n x (n+1)
# associated with a linear system.  RowReduction returns B, a NumPy array that
# represents the row echelon form of A.  RowReduction may not return correct
# results if the the matrix A does not have a pivot in each column.
# =============================================================================

    m = A.shape[0]  # A has m rows
    n = A.shape[1]  # It is assumed that A has m+1 columns

    B = np.copy(A).astype('float64')

    # For each step of elimination, we find a suitable pivot, move it into
    # position and create zeros for all entries below.

    for k in range(m):
        # Set pivot as (k,k) entry
        pivot = B[k][k]
        pivot_row = k

        # Find a suitable pivot if the (k,k) entry is zero
        while(pivot == 0 and pivot_row < m-1):
            pivot_row += 1
            pivot = B[pivot_row][k]

        # Swap row if needed
        if (pivot_row != k):
            B = lag.RowSwap(B,k,pivot_row)

        # If pivot is nonzero, carry on with elimination in column k
        if (pivot != 0):
            B = lag.RowScale(B,k,1./B[k][k])
            for i in range(k+1,m):
                B = lag.RowAdd(B,k,i,-B[i][k])
        else:
            print("Pivot could not be found in column",k,".")

    return B

Perhatikan bahwa dalam modul ini kita menggunakan operasi baris yang ditulis sebelumnya. Karena fungsi-fungsi tersebut tidak ditulis dalam *notebook ini*, kita perlu mengimpornya dari modul $\texttt{laguide}$.

Mari kita uji modul pada array matrik acak. Jalankan kode progam pada beberapa matriks acak dengan ukuran matrik sembarang. Apakah  selalu berhasil? Apakah Anda melihat adanya hasil yang tidak seperti biasanya? Apakah itu tergantung pada ukuran atau bentuknya? Apakah itu tergantung pada rentang angka yang digunakan?

In [ ]:
 ## Coba RowReduction disini.

Jika Anda sering menjalankan pengujian, kemungkinan besar Anda akan menemukan contoh yang hasilnya terlihat sedikit berbeda. Inilah salah satu kasusnya.

In [ ]:
NumericalReductionExample=np.array([[7,-6,6,-8],[-3,-5,-7,2],[1,-4,-7,-6],[-1,0,-2,-8]])
reduction_result = RowReduction(NumericalReductionExample)
print(reduction_result)

[[ 1.00000000e+00 -8.57142857e-01  8.57142857e-01 -1.14285714e+00]
 [-0.00000000e+00  1.00000000e+00  5.84905660e-01  1.88679245e-01]
 [-0.00000000e+00 -0.00000000e+00  1.00000000e+00  7.08463950e-01]
 [-0.00000000e+00 -0.00000000e+00  1.30206303e-17  1.00000000e+00]]


Ada dua hal yang kita amati dalam contoh ini. Yang pertama dan paling jelas adalah semua entri ditampilkan dalam notasi ilmiah. Jika kita perhatikan adalah bahwa hasilnya tidak persis seperti yang kita inginkan. Proses eliminasi seharusnya menghasilkan angka nol untuk semua entri di bawah diagonal utama, namun dalam kasus ini ada satu entri yang bukan $\texttt{0.000}$. Sebaliknya, jumlahnya sangat kecil, mendekati $10^{-17}$.

Sehingga perlu mempertanyakan kode program tersebut dan  mencari kesalahan, namun masalahnya di sini bukan terletak pada kode programmnya. Permasalahannya di sini adalah sesuatu yang lebih mendasar, dan melibatkan cara angka direpresentasikan dalam perhitungan dan batasan presisi komputer. Misalnya, bilangan $1/3$, yang memiliki representasi desimal yang terdiri dari string tak terhingga $3$s, tidak dapat direpresentasikan *tepat* menggunakan representasi desimal berhingga. Ini berarti bahwa array yang mewakili sistem mungkin sedikit salah bahkan sebelum penghitungan apa pun dilakukan. Saat komputer menjalankan operasi aritmatika, hasilnya harus dibulatkan menjadi angka-angka yang dapat direpresentasikan secara tepat. Ini yang  dikenal sebagai **roundoff error** dan inilah alasan kita tidak mendapatkan angka nol tepat untuk semua entri di bawah diagonal.

Dalam contoh ini, kita dapat melihat bahwa elemen-elemen yang seharusnya nol ternyata hanya sangat mendekati nol. Jika kita ingin menampilkan hasil dalam format lebih mudah dibaca, kita dapat menggunakan fungsi NumPy $\texttt{round}$ untuk membulatkan semua elemen ke jumlah digit tertentu.

In [ ]:
rounded_result = np.round(reduction_result,6)
print(rounded_result)

[[ 1.       -0.857143  0.857143 -1.142857]
 [-0.        1.        0.584906  0.188679]
 [-0.       -0.        1.        0.708464]
 [-0.       -0.        0.        1.      ]]


Catatan, kita sebaiknya  menggunakan hasil yang dibulatkan untuk tujuan menampilkan hasil. Jika kita melanjutkan dan menyelesaikan suatu sistem, atau melakukan perhitungan lain, kita harus menggunakan hasil asli dan bukan versi yang dibulatkan.

Kesalahan pembulatan dapat menghadirkan tantangan yang signifikan jika kita bekerja dengan array yang besar, dan kesalahan dibiarkan terakumulasi dan bertambah. Ada beberapa strategi yang dapat diterapkan untuk mengurangi kesalahan ini, dan memastikan bahwa hasil yang bermanfaat dapat diperoleh. Perangkat lunak apa pun  harus memberikan hasil yang dapat diandalkan yang harus memperhitungkan kesalahan pembulatan. Kami akan melanjutkan  eliminasi sederhana  dengan menyadari bahwa hasil yang di peroleh tidak selalu *tepat*.

### Fungsi untuk penyelesaian sistem persamaan

Sekarang kita dapat menggabungkan fungsi $\texttt{RowReduction}$ dan $\texttt{BackSubstitution}$ bersama-sama untuk menyelesaikan  sistem $AX=B$. Fungsi yang akan dibuat  akan akan menerima masukan  $A$ dan $B$, dan fungsi tersebut akan menghasilkan $X$. Berikut langkah-langkah yang perlu diselesaikan.

1. Membuat matriks augmented dari sistem persamaan yang ada.
2. Implementasikan $\texttt{RowReduction}$.
3. Pisahkan matriksnya.
4. Implementasikan $\texttt{BackSubstitution}$ dan keluarkan hasilnya.

Perhatikan bahwa ada cara lain untuk membuat fungsi. Misalnya, kita dapat meminta pengguna untuk memasukkan  matriks augmented, berarti pengguna harus melakukan langkah 1 setiap kali  menggunakan fungsi ini. Lebih baik membiarkan fungsi yang menyelesaikan langkah itu.

In [ ]:
def SolveSystem(A,B):
    # =============================================================================
    # A is a NumPy array that represents a matrix of dimension n x n.
    # B is a NumPy array that represents a matrix of dimension n x 1.
    # SolveSystem returns a NumPy array of dimension n x 1 such that AX = B.
    # If the system AX = B does not have a unique solution, SolveSystem may not
    # generate correct results.
    # =============================================================================

    # Check shape of A
    if (A.shape[0] != A.shape[1]):
        print("SolveSystem accepts only square arrays.")
        return
    n = A.shape[0]  # n is number of rows and columns in A

    # 1. Join A and B to make the augmented matrix
    A_augmented = np.hstack((A,B))

    # 2. Carry out elimination
    R = RowReduction(A_augmented)

    # 3. Split R back to nxn piece and nx1 piece
    B_reduced = R[:,n:n+1]
    A_reduced = R[:,0:n]

    # 4. Do back substitution
    X = BackSubstitution(A_reduced,B_reduced)
    return X

Mari kita uji modul dengan membuat matriks, menentukan solusinya sesunggunya, dan membangun sistem $AX=B$ sehingga kita mengetahui solusinya.

In [ ]:
A = np.array([[1,2,3],[0,1,-2],[3,3,-2]])
# We will choose the solution X_true
X_true = np.array([[1],[1],[1]])

# Now make B so that the solution to AX=B is X_true
B = A@X_true
print(B,'\n')
X = SolveSystem(A,B)
print(X)

[[ 6]
 [-1]
 [ 4]] 

[[1.]
 [1.]
 [1.]]


Selanjutnya, kita memodifikasi beberapa baris untuk menghasilkan sistem acak berikut dengan penyelesaiannya. Kita akan menggunakan $\texttt{SolveSystem}$ untuk menemukan penyelesaian dan selanjutnya menghitung selisih antara hasil dan penyelesaian sebenarnya .

In [ ]:
A = np.random.randint(-8,8,size=(4,4))
# We set X_true for a random solution
X_true = np.random.randint(-8,8,size=(4,1))
# Now make B so that the solution to AX=B is X_true
B = A@X_true
X = SolveSystem(A,B)
# Print the difference in computed solution and actual solution
print(X_true-X)

[[-8.88178420e-16]
 [ 0.00000000e+00]
 [-8.88178420e-16]
 [ 1.33226763e-15]]


**Latihan 1:** Gunakan $\texttt{np.random.rand(n,n)}$ untuk membuat  matriks koefisien dengan elemen-elemen matrik bilangan riel acakk. Buat sistem persamaan linier dengan penyelesaian yang diketahui menggunakan matriks ini, dan uji $\texttt{SolveSystem}$.

In [ ]:
## Koding penyelesaian disini

**Latihan 2:** Bereksperimenlah untuk melihat apa yang salah menggunakan $\texttt{RowReduction}$ pada matriks yang *bukan* $n\times (n+1)$.

In [ ]:
## Koding penyelesaian disini